# Example 1: Training SAE Model

This notebook demonstrates how to:
1. Load a language model and dataset
2. Save activations from a specific layer
3. Train a Sparse Autoencoder (SAE) on those activations
4. Save the trained SAE model

The trained SAE will be saved to `outputs/sae_model.pt` for use in the next example.


In [ ]:
# Setup and imports
%load_ext autoreload
%autoreload 2

import torch
from pathlib import Path
from datetime import datetime

from amber.store import LocalStore
from amber.adapters.text_snippet_dataset import TextSnippetDataset
from amber.core.language_model import LanguageModel
from amber.mechanistic.autoencoder.autoencoder import Autoencoder
from amber.mechanistic.autoencoder.train import SAETrainer, SAETrainingConfig

print("✅ Imports completed")


In [7]:
# Configuration
MODEL_ID = "sshleifer/tiny-gpt2"  # Small model for quick experimentation
HF_DATASET = "roneneldan/TinyStories"
DATA_SPLIT = "train"
TEXT_FIELD = "text"
DATA_LIMIT = 1000  # Number of text samples to use
MAX_LENGTH = 64    # Maximum sequence length
BATCH_SIZE_SAVE = 16  # Batch size for saving activations
BATCH_SIZE_TRAIN = 32  # Batch size for SAE training

# Choose which layer to hook - you can inspect available layers with model.layers.print_layer_names()
LAYER_SIGNATURE = 'gpt2lmheadmodel_transformer_h_0_attn_c_attn'  # Attention layer (better activations)

# Storage locations
CACHE_DIR = Path("outputs/cache")
STORE_DIR = Path("outputs/store")
RUN_ID = f"sae_training_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
SAE_MODEL_PATH = Path("outputs/sae_model.pt")

# Device configuration
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.float16 if torch.cuda.is_available() else None  # Use half precision on GPU

print("🚀 Starting SAE Training Example")
print(f"📱 Using device: {DEVICE}")
print(f"🔧 Model: {MODEL_ID}")
print(f"📊 Dataset: {HF_DATASET}")
print(f"🎯 Target layer: {LAYER_SIGNATURE}")
print()

# Create output directories
CACHE_DIR.mkdir(parents=True, exist_ok=True)
STORE_DIR.mkdir(parents=True, exist_ok=True)
print("✅ Output directories created")


🚀 Starting SAE Training Example
📱 Using device: cpu
🔧 Model: sshleifer/tiny-gpt2
📊 Dataset: roneneldan/TinyStories
🎯 Target layer: gpt2lmheadmodel_transformer_h_0_attn_c_attn

✅ Output directories created


In [8]:
# Step 1: Load language model with context
print("📥 Loading language model...")


# Load model using context
lm = LanguageModel.from_huggingface(MODEL_ID)
lm.model.to(DEVICE)

# Print available layers for reference
print("🔍 Available layers:")
lm.layers.print_layer_names()
print(f"✅ Model loaded: {lm.model_id}")
print(f"📱 Device: {DEVICE}")


📥 Loading language model...
🔍 Available layers:
gpt2lmheadmodel_transformer: No weight
gpt2lmheadmodel_transformer_wte: torch.Size([50257, 2])
gpt2lmheadmodel_transformer_wpe: torch.Size([1024, 2])
gpt2lmheadmodel_transformer_drop: No weight
gpt2lmheadmodel_transformer_h: No weight
gpt2lmheadmodel_transformer_h_0: No weight
gpt2lmheadmodel_transformer_h_0_ln_1: torch.Size([2])
gpt2lmheadmodel_transformer_h_0_attn: No weight
gpt2lmheadmodel_transformer_h_0_attn_c_attn: torch.Size([2, 6])
gpt2lmheadmodel_transformer_h_0_attn_c_proj: torch.Size([2, 2])
gpt2lmheadmodel_transformer_h_0_attn_attn_dropout: No weight
gpt2lmheadmodel_transformer_h_0_attn_resid_dropout: No weight
gpt2lmheadmodel_transformer_h_0_ln_2: torch.Size([2])
gpt2lmheadmodel_transformer_h_0_mlp: No weight
gpt2lmheadmodel_transformer_h_0_mlp_c_fc: torch.Size([2, 8])
gpt2lmheadmodel_transformer_h_0_mlp_c_proj: torch.Size([8, 2])
gpt2lmheadmodel_transformer_h_0_mlp_act: No weight
gpt2lmheadmodel_transformer_h_0_mlp_dropout: 

In [9]:
# Step 2: Load dataset
print("📥 Loading dataset...")
dataset = TextSnippetDataset.from_huggingface(
    HF_DATASET,
    split=DATA_SPLIT,
    cache_dir=str(CACHE_DIR),
    text_field=TEXT_FIELD,
    limit=DATA_LIMIT,
)
print(f"✅ Loaded {len(dataset)} text samples")


📥 Loading dataset...


Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 487313.12 examples/s]

✅ Loaded 1000 text samples


In [10]:
# Step 3: Save activations
print("💾 Saving activations...")
store = LocalStore(STORE_DIR)

lm.activations.infer_and_save(
    dataset,
    layer_signature=LAYER_SIGNATURE,
    run_name=RUN_ID,
    store=store,
    batch_size=BATCH_SIZE_SAVE,
    autocast=False,  # Disable autocast for consistency
)

# Verify activations were saved
batches = store.list_run_batches(RUN_ID)
print(f"✅ Saved {len(batches)} batches of activations")
print(f"📁 Run ID: {RUN_ID}")


💾 Saving activations...
✅ Saved 63 batches of activations
📁 Run ID: sae_training_20251028_234300


In [ ]:
# Step 4: Create SAE model
print("🏗️ Creating SAE model...")

# Get the hidden dimension from the first batch
first_batch = store.get_run_batch(RUN_ID, 0)
if isinstance(first_batch, dict):
    activations = first_batch["activations"]
else:
    activations = first_batch[0]  # Assume first tensor is activations

hidden_dim = activations.shape[-1]  # Last dimension is hidden size
print(f"📏 Hidden dimension: {hidden_dim}")

# Create SAE directly (context is created internally)
sae = Autoencoder(
    n_latents=hidden_dim * 4,  # 4x expansion factor
    n_inputs=hidden_dim,
    activation="TopK_8",  # TopK with k=8 for sparsity
    tied=False,  # Untied weights for better reconstruction
    init_method="kaiming",
    device=DEVICE,
)

print(f"🧠 SAE architecture: {hidden_dim} → {sae.context.n_latents} → {hidden_dim}")
sae.context.experiment_name = "sae_training"
sae.context.run_id = RUN_ID
print(f"🔧 Context: {sae.context.experiment_name}/{sae.context.run_id}")


🏗️ Creating SAE model...
📏 Hidden dimension: 6
🧠 SAE architecture: 6 → 24 → 6
🔧 Context: sae_training/sae_training_20251028_234300


In [12]:
# Step 5: Train SAE
print("🏋️ Training SAE...")

config = SAETrainingConfig(
    epochs=10,
    batch_size=BATCH_SIZE_TRAIN,
    lr=1e-3,
    l1_lambda=1e-4,  # L1 sparsity penalty
    device=DEVICE,
    max_batches_per_epoch=50,  # Limit batches per epoch for quick training
    project_decoder_grads=True,  # Project gradients for stability
    renorm_decoder_every=5,  # Renormalize decoder weights
    verbose=True,  # Enable progress logging
)

trainer = SAETrainer(sae, store, RUN_ID, config)
history = trainer.train()

print("✅ Training completed!")
print(f"📈 Final loss: {history['loss'][-1]:.6f}")
print(f"📈 Final reconstruction MSE: {history['recon_mse'][-1]:.6f}")
print(f"📈 Final L1 penalty: {history['l1'][-1]:.6f}")


🏋️ Training SAE...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2025-10-28 23:43:10,498 [INFO] amber.mechanistic.autoencoder.train: [SAETrainer] device=cpu dtype=None use_amp=True grad_accum_steps=1 batch_size=32 lr=0.001
2025-10-28 23:43:10,498 [INFO] amber.mechanistic.autoencoder.train: [SAETrainer] Starting training run_id=sae_training_20251028_234300 epochs=10 batch_size=32
Epochs: 100%|██████████| 10/10 [00:

✅ Training completed!
📈 Final loss: 0.000001
📈 Final reconstruction MSE: 0.000001
📈 Final L1 penalty: 0.006089


In [ ]:
# Step 6: Save trained SAE
print("💾 Saving trained SAE...")

# Add metadata about the training
metadata = {
    "dataset": HF_DATASET,
    "data_limit": DATA_LIMIT,
    "hidden_dim": hidden_dim,
    "n_latents": sae.context.n_latents,
    "activation": "TopK_8",
    "training_config": {
        "epochs": config.epochs,
        "batch_size": config.batch_size,
        "lr": config.lr,
        "l1_lambda": config.l1_lambda,
    },
    "run_id": RUN_ID,
    "training_history": history,
}

sae.save(
    name="sae_model",
    path=SAE_MODEL_PATH.parent,
    run_metadata=metadata,
    layer_signature=LAYER_SIGNATURE,
    model_id=MODEL_ID,
)

print(f"✅ SAE saved to: {SAE_MODEL_PATH}")


💾 Saving trained SAE...
✅ SAE saved to: outputs/sae_model.pt


In [ ]:
# Step 7: Save run metadata for next example
import json

run_metadata = {
    "run_id": RUN_ID,
    "layer_signature": LAYER_SIGNATURE,
    "hidden_dim": hidden_dim,
    "n_latents": sae.context.n_latents,
    "model_id": MODEL_ID,
    "dataset": HF_DATASET,
    "data_limit": DATA_LIMIT,
    "sae_model_path": str(SAE_MODEL_PATH),
    "store_dir": str(STORE_DIR),
    "cache_dir": str(CACHE_DIR),
}

metadata_path = Path("outputs/training_metadata.json")
with open(metadata_path, "w") as f:
    json.dump(run_metadata, f, indent=2)

print(f"📋 Training metadata saved to: {metadata_path}")
print()
print("🎉 SAE training completed successfully!")
print("📝 Next: Run 02_attach_sae_and_save_texts.ipynb to attach the SAE and collect top texts")


📋 Training metadata saved to: outputs/training_metadata.json

🎉 SAE training completed successfully!
📝 Next: Run 02_attach_sae_and_save_texts.ipynb to attach the SAE and collect top texts
